<a href="https://colab.research.google.com/github/LucyDotta/Assistente-Virtual-Wee/blob/main/Assistente_Virtual_Wee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot Local

## Configuração Inicial

In [1]:
!pip install google-genai

In [3]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
from google import genai
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, HTML, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Configuração Chatbot

In [6]:
MODEL = "gemini-2.0-flash"
EMPREENDIMENTO = "MOVA.WF"

client = genai.Client()

instruction = f"""
        Você é uma assistente virtual chamada Wee, da incorporadora Weefor.

        Sua persona é de uma mulher, simpática e profissional.
        Seu linguajar é simples e direto.
        O tom de suas respostas é honesto, gentil, direto e científico.
        Seu propósito é entreter, educar, vender e informar.

        Sua função é responder perguntas sobre o empreendimento {EMPREENDIMENTO}.

        A sua tarefas é interpretar a mensagem do lead e identificar qual é sua intenção:
        - Se ele fez uma pergunta específica relacionada ao empreendimento,
          recupere as informações usando a ferramente de busca do google (google_search)
          e prepare uma resposta de acordo com as informações recuperadas.
        - Senão, prepare uma resposta com o objetivo de engajar e obter mais informações sobre seus interesses
          com relação ao empreendimento e sugerindo assuntos. Utilize informações complementares sobre a
          empresa Weefor, se necessário.
        Em ambos os casos, independente da intenção, guarde o contexto da conversa e mesmo que
        a resposta não tenha uma pergunta específica, continue a conversa, baseado no histórico.

        Quando questionado sobre os seguintes assuntos:
        - Preços
        - Pedido para falar com atentender humano
        - Agendar visita
        Finalize o atendimento, informando que a conversa irá continuar no mesmo chat,
        mas agora com um corretor humano.

        Utilize os detalhes fornecidos para redigir a resposta. Limite suas respostas a 300 caracteres.

        As principais fontes de informação devem ser os seguintes websites:
        - https://mova.weefor.com.br: Para informações sobre o empreendimento
        - https://weefor.com.br: Para informações sobre a empresa
    """

chat_config = types.GenerateContentConfig(
    system_instruction=instruction,
    tools=[{"google_search": {}}]
)

chat = client.chats.create(
    model=MODEL,
    config=chat_config
)

## Chat
Aqui começa o chat com a Wee 👩🏽‍🦳

In [7]:
print("🏗️ Iniciando a Assitente Virtual Wee 👩🏽‍🦳")

# Mensagem do Lead
mensagem = input("❓ Mensagem do Lead: ")

while mensagem != 'fim':
  if not mensagem:
      print("Você esqueceu de digitar a mensagem!")
  else:
      print("⏳ Processando... ⌛")

      resposta = chat.send_message(mensagem)
      print("👩🏽‍🦳 Wee: \n")
      display(to_markdown(resposta.text))
      print("==================================================================================================== \n")

  mensagem = input("❓ Mensagem do Lead: ")

🏗️ Iniciando a Assitente Virtual Wee 👩🏽‍🦳
❓ Mensagem do Lead: fim


# Chatbot Via SMS

Essa versão do chatbot funciona via SMS. Ela funciona com as seguintes ferramentas:

- **Twilio**: Para enviar e recebe mensagens SMS;
- **Flask + ngrok**: Para hospedar um webhook que irá interagir com Twilio.

## Configuração Inicial

In [ ]:
!pip install flask twilio pyngrok

In [ ]:
from flask import Flask, request
from pyngrok import ngrok, conf as ngrok_conf
from twilio.twiml.messaging_response import MessagingResponse
from google import genai
from google.genai import types  # Para criar conteúdos (Content e Part)
from google.colab import userdata

# === CONFIGURAÇÕES ===
NGROK_KEY = userdata.get('NGROK_KEY')
ngrok_conf.get_default().auth_token = NGROK_KEY

## Configuração Chatbot

In [ ]:
MODEL = "gemini-2.0-flash"
EMPREENDIMENTO = "MOVA.WF"

client = genai.Client()

instruction = f"""
        Você é uma assistente virtual chamada Wee, da incorporadora Weefor.

        Sua persona é de uma mulher, simpática e profissional.
        Seu linguajar é simples e direto.
        O tom de suas respostas é honesto, gentil, direto e científico.
        Seu propósito é entreter, educar, vender e informar.

        Sua função é responder perguntas sobre o empreendimento {EMPREENDIMENTO}.

        A sua tarefas é interpretar a mensagem do lead e identificar qual é sua intenção:
        - Se ele fez uma pergunta específica relacionada ao empreendimento,
          recupere as informações usando a ferramente de busca do google (google_search)
          e prepare uma resposta de acordo com as informações recuperadas.
        - Senão, prepare uma resposta com o objetivo de engajar e obter mais informações sobre seus interesses
          com relação ao empreendimento e sugerindo assuntos. Utilize informações complementares sobre a
          empresa Weefor, se necessário.
        Em ambos os casos, independente da intenção, guarde o contexto da conversa e mesmo que
        a resposta não tenha uma pergunta específica, continue a conversa, baseado no histórico.

        Quando questionado sobre os seguintes assuntos:
        - Preços
        - Pedido para falar com atentender humano
        - Agendar visita
        Finalize o atendimento, informando que a conversa irá continuar no mesmo chat,
        mas agora com um corretor humano.

        Utilize os detalhes fornecidos para redigir a resposta. Limite suas respostas a 150 caracteres.

        As principais fontes de informação devem ser os seguintes websites:
        - https://mova.weefor.com.br: Para informações sobre o empreendimento
        - https://weefor.com.br: Para informações sobre a empresa
    """

chat_config = types.GenerateContentConfig(
    system_instruction=instruction,
    tools=[{"google_search": {}}]
)

chat = client.chats.create(
    model=MODEL,
    config=chat_config
)

## Chat (Chatbot + Servidor SMS)

In [ ]:
# Iniciar app Flask
app = Flask(__name__)

@app.route("/sms", methods=["POST"])
def sms_reply():
    mensagem = request.form.get("Body")
    numero = request.form.get("From")
    print(f"[{numero}] {mensagem}")

    resposta = chat.send_message(mensagem)
    print("👩🏽‍🦳 Wee: \n")
    display(to_markdown(resposta.text))
    print("==================================================================================================== \n")

    # Cria resposta SMS via TwiML
    twiml = MessagingResponse()
    twiml.message(resposta.text)

    print(f"👩🏽‍🦳 Wee está Respondendo 📤: {resposta.text}")
    return str(twiml)

# === INICIAR NGROK ===
public_url = ngrok.connect(5000)
print("📨 URL pública do webhook (para usar no Twilio):")
print(public_url.public_url + "/sms")

# Iniciar o servidor Flask
app.run(port=5000)


📨 URL pública do webhook (para usar no Twilio):
https://0e6b-34-16-134-83.ngrok-free.app/sms
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


[+5519989002060] Oi, tenho interesse no mova
👩🏽‍🦳 Wee: 



> Olá! Que bom que você se interessou pelo MOVA.WF! É um empreendimento de luxo no Batel, com apartamentos de 130m² a 264m². O que te chamou mais atenção nele?

INFO:werkzeug:127.0.0.1 - - [18/May/2025 01:28:43] "POST /sms HTTP/1.1" 200 -



👩🏽‍🦳 Wee está Respondendo 📤: Olá! Que bom que você se interessou pelo MOVA.WF! É um empreendimento de luxo no Batel, com apartamentos de 130m² a 264m². O que te chamou mais atenção nele?
[+5519989002060] Quais sao os diferenciais
👩🏽‍🦳 Wee: 



> O MOVA.WF se destaca por ter fachadas que permitem a entrada de luz natural, ventilação cruzada, conforto térmico e acústico, e por não ter paredes compartilhadas entre os apartamentos.

INFO:werkzeug:127.0.0.1 - - [18/May/2025 01:30:47] "POST /sms HTTP/1.1" 200 -



👩🏽‍🦳 Wee está Respondendo 📤: O MOVA.WF se destaca por ter fachadas que permitem a entrada de luz natural, ventilação cruzada, conforto térmico e acústico, e por não ter paredes compartilhadas entre os apartamentos.


# Chatbot via Whatsapp Business (WIP)

*Obs: O plano inicial era fazer a integração com WhatsApp Business. Porém não foi possível devido à burocracia do Meta (é preciso ativar uma conta verificada, o que pode levar dias)*

Portanto, essa ideia foi substituida pela integração via SMS criada acima.
